In [1]:
import pandas
import openai
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import slack_alarm
import requests

# slack 알람 사용법
# slack_alarm.send_notification("CSV 파일 처리")

In [2]:
# 데이터 확인
## 100개까지만 만든 dummy data
df = pd.read_csv("dummy_dataset.csv").drop(columns = 'index')
df_columns_sort = df[['text', 'user_data', 'detail', 'contents', 'ERD_data', 'relationships_data', 'API_specs_data']]
df_columns_sort.head()

,text,user_data,detail,contents,ERD_data,relationships_data,API_specs_data
0,"GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹...","[{'projectName': 'StudyHub', 'projectTarget': ...","[{'requirementType': 'FUNCTIONAL1', 'content':...","{'project_summary': {'title': 'GQ 서비스', 'categ...","[{'name': 'User', 'attributes': [{'name': 'use...","[{'from': 'User', 'to': 'StudyGroup', 'type': ...","{'api_specification': {'openapi': '3.0.0', 'in..."
1,‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용...,"[{'projectName': '기억의 다리', 'projectTarget': '치...","[{'requirementType': 'FUNCTIONAL1', 'content':...","{'project_summary': {'title': '기억박물관', 'catego...","[{'name': 'User', 'attributes': [{'name': 'use...","[{'from': 'User', 'to': 'Photo', 'type': 'one-...","{'api_specification': {'openapi': '3.0.0', 'in..."
2,"이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로...",[{'projectName': 'EasyAccess Subway Navigator'...,"[{'requirementType': 'FUNCTIONAL1', 'content':...",{'project_summary': {'title': '보행 약자를 위한 지하철 길...,"[{'name': 'User', 'attributes': [{'name': 'use...","[{'from': 'User', 'to': 'Route', 'type': 'one-...","{'api_specification': {'openapi': '3.0.0', 'in..."
3,"이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있...","[{'projectName': '이웃사촌', 'projectTarget': '1인 ...","[{'requirementType': 'FUNCTIONAL1', 'content':...","{'project_summary': {'title': '1인 가구 맞춤형 플랫폼',...","[{'name': 'User', 'attributes': [{'name': 'use...","[{'from': 'User', 'to': 'Post', 'type': 'one-t...","{'api_specification': {'openapi': '3.0.0', 'in..."
4,"이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안...","[{'projectName': 'EmergencyCare Finder', 'proj...","[{'requirementType': 'FUNCTIONAL1', 'content':...",{'project_summary': {'title': '응급실 및 병원 찾기 웹앱'...,"[{'name': 'User', 'attributes': [{'name': 'use...","[{'from': 'User', 'to': 'EmergencyRoom', 'type...","{'api_specification': {'openapi': '3.0.0', 'in..."


In [5]:
df_top3 = df_columns_sort.head(1)
project_overview_lst = df_top3['user_data'].tolist()
existing_requirement_lst = df_top3['detail'].tolist()

content_lst = []
erd_lst = []
relationship_lst = []
API_spec_data_lst = []


project_overview = project_overview_lst[0]
existing_requirement = existing_requirement_lst[0]

# 요청 데이터
data = {
    "project_overview": project_overview,
    "requirements": existing_requirement,
    "max_tokens": 4000,
    "temperature": 0.3,
    "model": "ft:gpt-4o-mini-2024-07-18:test::BebIPMSD"
}

# API 호출
response = requests.post(
    "http://127.0.0.1:8000/api/pja/json_text/generate",  # 로컬 서버 주소
    json=data,
    headers={"Content-Type": "application/json"}
)

# 응답 처리
if response.status_code == 200:
    result = response.json()
    content_lst.append({'project_summary' : result['json']['project_summary']})
    erd_lst.append({'erd_talbes' : result['json']['erd_tables']})
    relationship_lst.append({'erd_relationships' : result['json']['erd_relationships']})
    API_spec_data_lst.append({ 'apiSpecifications' : result['json']['apiSpecifications']})

else:
    print(f"Error: {response.status_code}")
    print(response.text)
    slack_alarm.send_notification(f"{1}번쨰 오류 발생")

df_top3['contents'] = content_lst
df_top3['ERD_data'] = erd_lst
df_top3['relationships_data'] = relationship_lst
df_top3['API_specs_data'] = API_spec_data_lst

slack_alarm.send_notification("parsing")

C:\Users\mir96\AppData\Local\Temp\ipykernel_18288\262662817.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top3['contents'] = content_lst
C:\Users\mir96\AppData\Local\Temp\ipykernel_18288\262662817.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top3['ERD_data'] = erd_lst
C:\Users\mir96\AppData\Local\Temp\ipykernel_18288\262662817.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

True

In [9]:
print(df_top3['API_specs_data'][0])

{'apiSpecifications': [{'title': 'User Registration', 'tag': 'Users', 'path': '/api/users/register', 'http_method': 'POST', 'request': [{'field': 'email', 'type': 'string', 'example': 'user@example.com'}, {'field': 'password', 'type': 'string', 'example': 'securePassword123'}, {'field': 'username', 'type': 'string', 'example': 'study_user'}], 'response': [{'status_code': '201', 'message': 'User registered successfully', 'data': {'userId': 1, 'username': 'study_user'}}, {'status_code': '400', 'message': 'Invalid input data', 'data': None}]}, {'title': 'User Login', 'tag': 'Users', 'path': '/api/users/login', 'http_method': 'POST', 'request': [{'field': 'email', 'type': 'string', 'example': 'user@example.com'}, {'field': 'password', 'type': 'string', 'example': 'securePassword123'}], 'response': [{'status_code': '200', 'message': 'Login successful', 'data': {'token': 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9...', 'userId': 1, 'username': 'study_user'}}, {'status_code': '401', 'message': 'Inv

In [ ]:
df_top3.to_csv('dummy_data_utf-8.csv', encoding = 'utf-8', index = False)
df_top3.to_csv('dummy_data_euc-kr.csv', encoding = 'euc-kr', index = False)